In [ ]:
import sys
sys.path.append('..')

from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt

from arch import utils, metadata, hex, hist, transport, ela, filter, frequency, pca

In [ ]:
PATH = Path('data/face.jpg')
img = utils.load_image(PATH)

---

In [ ]:
md = {}
md.update(metadata.file_metadata(PATH))
md.update(metadata.exif_data(PATH))

[f'{l.upper()}: {md[l]}' for l in md]

In [ ]:
with open(PATH, 'rb') as image_file:
    data = image_file.read()
hex.hexdump(data, 512)

---

In [ ]:
pixel_count, stat = hist.process(img)

print(f"""
Image Spec: {img.shape}
Min: {img.min()} | Max: {img.max()}
Pixel count: {stat[0]}
Least Frequent: {stat[1]}
Most Frequent: {stat[2]}
Mean: {stat[3]}
Deviation: {stat[4]}
Median: {stat[5]}
Nonzero: {stat[6]}
Empty Bins: {stat[7]}
Unique Colours: {stat[8]}
""")

In [ ]:
mag, phase, fft = frequency.calc_fft(img)

In [ ]:
# Define radial distance
nx, ny = mag.shape
x = np.arange(-mag.shape[0]//2, mag.shape[0]//2)
y = np.arange(-mag.shape[1]//2, mag.shape[1]//2)
X, Y = np.meshgrid(x, y)
dist = np.sqrt(X**2 + Y**2)

# Compute spatial frequency
hist, bins = np.histogram(dist.flatten(), bins=np.arange(dist.max()), weights=mag.flatten())

count = hist[1:].argmax()
print('Freqency at which power is greatest:', count)

In [ ]:
plt.figure(figsize=(25,5))
width = 0.25
alpha = 0.5

# Luminance Distribution
plt.subplot(1,5,1)
plt.plot(pixel_count[0], pixel_count[1], lw=0.5)
plt.bar(pixel_count[0], pixel_count[1], color='k', width=width)

plt.vlines(pixel_count[1].argmax(), 0, pixel_count[1].max(), linewidth=width * 5, color='r', label='Max Luminance')

plt.title('Luminance Distribution')
plt.xlabel('Pixel Value'), plt.ylabel('Count')
plt.xlim([-1, 255]), plt.legend()
plt.grid(alpha=0.25), plt.tight_layout()

# Channel Distribution
plt.subplot(1,5,2)
plt.bar(pixel_count[0], pixel_count[2], color='r', width=width, label='Red Channel')
plt.bar(pixel_count[0], pixel_count[3], color='g', width=width, label='Green Channel')
plt.bar(pixel_count[0], pixel_count[4], color='b', width=width, label='Blue Channel')

plt.plot(pixel_count[0], pixel_count[2], 'r', lw=0.5)
plt.plot(pixel_count[0], pixel_count[3], 'g', lw=0.5)
plt.plot(pixel_count[0], pixel_count[4], 'b', lw=0.5)

plt.title('Channel Distribution')
plt.xlabel('Pixel Value'), plt.ylabel('Count')
plt.xlim([-1, 255]), plt.legend()
plt.grid(alpha=0.25), plt.tight_layout()

plt.subplot(1,5,3)
plt.imshow(np.log(mag), 'binary')
plt.title('Power Spectra')

plt.subplot(1,5,4)
plt.imshow(phase, 'binary')
plt.title('Phase')

plt.subplot(1,5,5)
plt.plot(bins[:-1], hist, lw=1)
plt.vlines(count+1, 0, hist[1:].max(), linewidth=1, color='r')
plt.title('Spatial Frequency')
plt.xlabel('Distance from Origin'), plt.ylabel('Power')
plt.xlim([1,int(dist.max())]), plt.ylim([0,int(hist[1:].max())])
plt.grid(alpha=0.25)
plt.tight_layout()

---

In [ ]:
ela = ela.error_level_analysis(PATH, quality=90)

---

In [ ]:
hsv = filter.rgb2hsv(img)
lum = filter.luminance_gradient(img, intensity=0.35)
echo = filter.echo_edge(img, radius=2, contrast=0.85)
noise = filter.noise_seperation(img, mode='median', radius=2, sigma=3)

In [ ]:
mask = frequency.mask_fft(fft, radius=2, blur=25)
low = frequency.decode_low(fft, mask)
high = frequency.decode_high(fft, mask)

In [ ]:
n_comp = pca.extract_components(img)
utils.contact_layer(n_comp, 3, 3).resize((1000,1000))

---

### WRITE

In [ ]:
aov = {'error_level': ela,
       'hsv': hsv,
       'luminance_gradient': lum,
       'echo_edge': echo,
       'noise_seperation': noise,
       'low_freq': low,
       'high_freq': high,
       }

header = transport.write(img, aov, 'test.exr')
header

---

### READ

In [ ]:
layers = transport.read('test.exr')

In [ ]:
label = [l[0] for l in layers]
data = [l[1] for l in layers]
utils.contact_layer(data, 2, 4, label)